<a href="https://colab.research.google.com/github/axiomhaidarcreate/project-AI-/blob/main/cass_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os

# ربط الدرايف
drive.mount('/content/drive')

# المسار اللي قلت عليه
data_path = "/content/drive/MyDrive/kt"

# عرض الملفات والفولدرات
if os.path.exists(data_path):
    print("📂 محتوى مجلد kt:")
    for f in os.listdir(data_path):
        print(f)
else:
    print("❌ المجلد غير موجود:", data_path)


Mounted at /content/drive
📂 محتوى مجلد kt:
running
handwaving
walking
boxing


In [ ]:
import os

data_path = "/content/drive/MyDrive/kt"

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        files = os.listdir(folder_path)
        print(f"\n📂 {folder} ({len(files)} فيديو)")
        for f in files[:10]:  # نعرض أول 10 فيديوهات فقط
            fpath = os.path.join(folder_path, f)
            size_mb = os.path.getsize(fpath) / (1024*1024)
            print(f"   - {f} ({size_mb:.2f} MB)")



📂 running (100 فيديو)
   - person01_running_d3_uncomp.avi (1.43 MB)
   - person02_running_d2_uncomp.avi (5.53 MB)
   - person01_running_d1_uncomp.avi (1.31 MB)
   - person01_running_d2_uncomp.avi (1.21 MB)
   - person02_running_d1_uncomp.avi (1.47 MB)
   - person01_running_d4_uncomp.avi (0.65 MB)
   - person04_running_d2_uncomp.avi (2.23 MB)
   - person03_running_d3_uncomp.avi (1.69 MB)
   - person04_running_d1_uncomp.avi (1.15 MB)
   - person03_running_d1_uncomp.avi (1.54 MB)

📂 handwaving (1 فيديو)
   - person01_handwaving_d2_uncomp.avi (2.78 MB)

📂 walking (98 فيديو)
   - person01_walking_d4_uncomp.avi (0.79 MB)
   - person01_walking_d2_uncomp.avi (2.45 MB)
   - person01_walking_d1_uncomp.avi (2.28 MB)
   - person01_walking_d3_uncomp.avi (3.67 MB)
   - person02_walking_d4_uncomp.avi (0.86 MB)
   - person02_walking_d3_uncomp.avi (2.11 MB)
   - person03_walking_d1_uncomp.avi (2.55 MB)
   - person02_walking_d1_uncomp.avi (1.96 MB)
   - person02_walking_d2_uncomp.avi (4.02 MB)
   - per

In [ ]:
!pip install --upgrade transformers
!pip install sentencepiece
!pip install opencv-python
!pip install torch torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 120.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import cv2
import os
from transformers import BlipProcessor, BlipForConditionalGeneration


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
blip_model.eval()  # وضع المودل في وضع التقييم


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [ ]:
class VideoDataset(Dataset):
    def __init__(self, root_dir, classes, transform=None, max_frames=16):
        self.root_dir = root_dir
        self.classes = classes
        self.transform = transform
        self.max_frames = max_frames
        self.samples = []

        for label, cls in enumerate(classes):
            cls_folder = os.path.join(root_dir, cls)
            for file in os.listdir(cls_folder):
                if file.endswith(".avi") or file.endswith(".mp4"):
                    self.samples.append((os.path.join(cls_folder, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        cap = cv2.VideoCapture(video_path)
        frames = []
        count = 0

        while count < self.max_frames:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if self.transform:
                frame = self.transform(frame)
            frames.append(frame)
            count += 1

        cap.release()

        if len(frames) == 0:
            # فيديو فارغ، نعيد فريم أسود
            frames.append(torch.zeros(3, 224, 224))

        # إذا عدد الفريمات أقل من max_frames، نكرر آخر فريم
        while len(frames) < self.max_frames:
            frames.append(frames[-1])

        video_tensor = torch.stack(frames)  # [max_frames, 3, H, W]
        return video_tensor, label


In [ ]:
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor()
])


In [ ]:
classes = ["running", "walking"]
dataset = VideoDataset("/content/drive/MyDrive/kt", classes, transform=transform)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)


In [ ]:
class BLIPClassifier(nn.Module):
    def __init__(self, blip_model, embedding_dim=768, num_classes=2):
        super().__init__()
        self.blip_model = blip_model.vision_model  # نستعمل فقط جزء الرؤية
        self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        # x: [batch, frames, 3, 224, 224]
        batch_size, num_frames, C, H, W = x.shape
        x = x.view(batch_size * num_frames, C, H, W)
        with torch.no_grad():
            features = self.blip_model(pixel_values=x).last_hidden_state.mean(dim=1)
        features = features.view(batch_size, num_frames, -1)
        features = features.mean(dim=1)  # متوسط الفريمات
        out = self.fc(features)
        return out


In [ ]:
model = BLIPClassifier(blip_model).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-4)  # فقط طبقة FC
epochs = 5


In [ ]:
for epoch in range(epochs):
    total_loss = 0
    for videos, labels in dataloader:
        videos, labels = videos.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(videos)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")


Epoch 1/5, Loss: 0.6970
Epoch 2/5, Loss: 0.6904
Epoch 3/5, Loss: 0.6844
Epoch 4/5, Loss: 0.6826
Epoch 5/5, Loss: 0.6752


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/blip_video_classifier.pth")
print("Saved model to Google Drive!")


Saved model to Google Drive!


In [ ]:
import gradio as gr
import torch
import cv2
from torchvision import transforms as T

# Classes
classes = ["running", "walking"]
device = "cuda" if torch.cuda.is_available() else "cpu"

# Transform
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224,224)),
    T.ToTensor()
])

# دالة تصنيف الفيديو
def classify_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    max_frames = 16
    count = 0
    while count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = transform(frame)
        frames.append(frame)
        count += 1
    cap.release()

    if len(frames) == 0:
        frames.append(torch.zeros(3,224,224))
    while len(frames) < max_frames:
        frames.append(frames[-1])

    video_tensor = torch.stack(frames).unsqueeze(0).to(device)
    with torch.no_grad():
        pred = model(video_tensor)
    return classes[pred.argmax().item()]

# واجهة Gradio الحديثة
iface = gr.Interface(
    fn=classify_video,
    inputs=gr.Video(label="Upload Video"),
    outputs=gr.Textbox(label="Predicted Class"),
    title="BLIP Video Classifier",
    description="Upload a video and the model will classify it as 'running' or 'walking'."
)

iface.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8cae91ae7a12dbe64d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8cae91ae7a12dbe64d.gradio.live
